1) The provided data (link above) contains various details and attributes associated with used cars. The target variable, which is the central focus of analysis, is the price of the used cars, and it is measured in lakhs. The data in this dataset is tabular, with rows and columns, where each row represents a specific used car listing, and each column represents a particular attribute or feature of these cars.  Features are Make and model of the car, Location or city of sale, Year of manufacture, Mileage, Odometer (kilometers driven), Fuel type (petrol or diesel), Transmission type (manual or automatic), Number of owners, Engine displacement, Engine horsepower, Number of seats, and Price when the car was new.

Use this data to perform the following:

a)  Look for the missing values in all the columns and either impute them (replace with mean, median, or mode) or drop them. Justify your action for this task.     (4 points)  

b) Remove the units from some of the attributes and only keep the numerical values
  1. Remove kmpl from “Mileage”
  2. Remove CC from “Engine”
  3. Remove bhp from “Power”
  4. Remove lakh from “New_price”

C) Change the categorical variables (“Fuel_Type” and “Transmission”) into numerical one hot encoded value.  (4 points).  


d) Create one more feature and add this column to the dataset (you can use mutate function in R for this). For example, you can calculate the current age of the car by subtracting “Year” value from the current year.   (4 points)   

e) Perform select, filter, rename, mutate, arrange and summarize with group by operations (or their equivalent operations in python) on this dataset. (4 points)  


In [108]:
# Install tidyverse if not already installed (usually pre-installed in Colab R runtime)
install.packages("tidyverse")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [109]:
# Install and load the mltools package
install.packages("mltools")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [110]:
library(tibble)  #Used for ease of Column management
library(tidyverse)  #Used for ease of Column management
library(mltools)
library(data.table) # one_hot requires data.table

source_file_url = "https://raw.githubusercontent.com/joelvinas/COMP-SCI_5530/refs/heads/main/Assignment_2/Data/train.csv"
df <- read.csv(source_file_url)

#attach(df)

# Get current column names
#current_cols <- names(df)

In [111]:
head(df)

,X,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5,,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7,,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5,,17.74
5,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5,,3.50
6,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8,21 Lakh,17.50


In [112]:
#b.1: Handle Mileage

#Sources:
  #https://www.engineeringtoolbox.com/fuels-densities-specific-volumes-d_166.html
  #https://www.sciencedirect.com/topics/engineering/diesel-fuel
  #https://www.transportpolicy.net/standard/india-fuels-diesel-and-gasoline/

# Initialize Mileage_kmkg with NA_real_ (for numeric NAs)
# This will hold the numeric mileage values
df$Mileage_kmkg <- NA_real_

# Process 'km/kg' values: extract numeric part, convert to numeric
# This applies to all rows where 'Mileage' contains 'km/kg'
km_kg_indices <- grepl("km/kg", df$Mileage, fixed = TRUE)
df$Mileage_kmkg[km_kg_indices] <- suppressWarnings(as.numeric(gsub(" km/kg", "", df$Mileage[km_kg_indices], fixed = TRUE)))

# Process 'kmpl' values:

#   Handle Diesel
density_diesel <- 0.85 #kg/l
kmpl_diesel_indices <- grepl("kmpl", df$Mileage, fixed = TRUE) & df$Fuel_Type == "Diesel"
df$Mileage_kmkg[kmpl_diesel_indices] <- suppressWarnings(as.numeric(gsub(" kmpl", "", df$Mileage[kmpl_diesel_indices], fixed = TRUE)) / density_diesel)

#   Handle Petrol (Gasoline)
density_petrol <- 0.75 #kg/l
kmpl_petrol_indices <- grepl("kmpl", df$Mileage, fixed = TRUE) & df$Fuel_Type == "Petrol"
df$Mileage_kmkg[kmpl_petrol_indices] <- suppressWarnings(as.numeric(gsub(" kmpl", "", df$Mileage[kmpl_petrol_indices], fixed = TRUE)) / density_petrol)

df$Mileage <- NULL #Remove the Mileage Column
#filtered_df <- df %>%
#  filter(is.na(Mileage_kmkg))

In [113]:
#b.2: Handle Engine
df <- df %>%
  mutate(Engine_CC = as.numeric(str_remove_all(Engine, " CC")))

df$Engine <- NULL #Remove the Engine Column
#Note that null values will be stored as NA

# filtered_df <- df %>%
#   filter(is.na(Engine_CC))
# head(filtered_df)

In [114]:
#b.3: Handle Power
df <- df %>%
  mutate(Power_bhp = as.numeric(str_remove_all(Power, " bhp")))

df$Power <- NULL  #Remove the Power Column

In [115]:
#b.3: Handle the New Price
# Ensure 'New_Price' is treated as character for string operations
df$New_Price <- as.character(df$New_Price)

# Initialize NewPriceLakh column with NA_real_ (for numeric NAs)
df$NewPriceLakh <- NA_real_

# Process 'Cr' values: extract numeric part, convert to numeric, multiply by 100
df$NewPriceLakh[grepl("Cr", df$New_Price, fixed = TRUE)] <- suppressWarnings(as.numeric(gsub(" Cr", "", df$New_Price[grepl("Cr", df$New_Price, fixed = TRUE)], fixed = TRUE)) * 100)

# Process 'Lakh' values: extract numeric part, convert to numeric
df$NewPriceLakh[grepl("Lakh", df$New_Price, fixed = TRUE)] <- suppressWarnings(as.numeric(gsub(" Lakh", "", df$New_Price[grepl("Lakh", df$New_Price, fixed = TRUE)], fixed = TRUE)))

# Remove the New_Price column
df$New_Price <- NULL


In [116]:
#C) Change the categorical variables (“Fuel_Type” and “Transmission”) into numerical one hot encoded value. (4 points).

#c.1: Fuel_Type
df$Fuel_Type <- as.character(df$Fuel_Type)  # Ensure 'Fuel_Type' is treated as character for string operations

# Initialize IsDiesel column with FALSE
df$IsFuelTypeDiesel <- FALSE
df$IsFuelTypeDiesel[df$Fuel_Type == 'Diesel'] <- TRUE

df$IsFuelTypePetrol <- FALSE
df$IsFuelTypePetrol[df$Fuel_Type == 'Petrol'] <- TRUE

df$Fuel_Type <- NULL  #Drop the Fuel_Type column

In [117]:
#C) Change the categorical variables (“Fuel_Type” and “Transmission”) into numerical one hot encoded value. (4 points).

#c.2: Transmission

df$Transmission <- as.character(df$Transmission)  # Ensure 'Transmission' is treated as character for string operations

# Initialize IsTransmissionManual column with FALSE
df$IsTransmissionManual <- FALSE
df$IsTransmissionManual[df$Transmission == 'Manual'] <- TRUE


# Initialize IsTransmissionAutomatic column with FALSE
df$IsTransmissionAutomatic <- FALSE
df$IsTransmissionAutomatic[df$Transmission == 'Automatic'] <- TRUE

df$Transmission <- NULL  #Drop the Transmission column

In [118]:
head(df)

,X,Name,Location,Year,Kilometers_Driven,Owner_Type,Seats,Price,Mileage_kmkg,Engine_CC,Power_bhp,NewPriceLakh,IsFuelTypeDiesel,IsFuelTypePetrol,IsTransmissionManual,IsTransmissionAutomatic
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,5,12.50,23.14118,1582,126.20,NA,TRUE,FALSE,TRUE,FALSE
2,2,Honda Jazz V,Chennai,2011,46000,First,5,4.50,13.00000,1199,88.70,8.61,FALSE,TRUE,TRUE,FALSE
3,3,Maruti Ertiga VDI,Chennai,2012,87000,First,7,6.00,24.43529,1248,88.76,NA,TRUE,FALSE,TRUE,FALSE
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,5,17.74,17.88235,1968,140.80,NA,TRUE,FALSE,FALSE,TRUE
5,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,5,3.50,27.15294,1461,63.10,NA,TRUE,FALSE,TRUE,FALSE
6,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,8,17.50,13.36471,2755,171.50,21.00,TRUE,FALSE,FALSE,TRUE


In [119]:
#d) Create one more feature and add this column to the dataset (you can use mutate function in R for this).
#For example, you can calculate the current age of the car by subtracting “Year” value from the current year. (4 points)

#Reference:
#https://unepccc.org/wp-content/uploads/2014/08/assessment-of-motor-vehicle-use-characteristics-in-three-indian-cities.pdf#:~:text=The%20estimated%20average%20annual%20mileage%20(%C2%B1%2095%25,km%20and%207%2C255%20%C2%B1%20325%20km%2C%20respectively.

avg_km_driven_yr <- 12000
df$PricePerSeat <- df$Price / df$Seats
df$EstYrsDriven <- df$Kilometers_Driven / avg_km_driven_yr
df$PricePerYr <- df$Price / (df$EstAnnualMileage)

head(df)

,X,Name,Location,Year,Kilometers_Driven,Owner_Type,Seats,Price,Mileage_kmkg,Engine_CC,Power_bhp,NewPriceLakh,IsFuelTypeDiesel,IsFuelTypePetrol,IsTransmissionManual,IsTransmissionAutomatic,PricePerSeat,EstAnnualMileage,PricePerYr
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,5,12.50,23.14118,1582,126.20,NA,TRUE,FALSE,TRUE,FALSE,2.5000000,3.416667,3.6585366
2,2,Honda Jazz V,Chennai,2011,46000,First,5,4.50,13.00000,1199,88.70,8.61,FALSE,TRUE,TRUE,FALSE,0.9000000,3.833333,1.1739130
3,3,Maruti Ertiga VDI,Chennai,2012,87000,First,7,6.00,24.43529,1248,88.76,NA,TRUE,FALSE,TRUE,FALSE,0.8571429,7.250000,0.8275862
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,5,17.74,17.88235,1968,140.80,NA,TRUE,FALSE,FALSE,TRUE,3.5480000,3.389167,5.2343251
5,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,5,3.50,27.15294,1461,63.10,NA,TRUE,FALSE,TRUE,FALSE,0.7000000,7.249917,0.4827642
6,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,8,17.50,13.36471,2755,171.50,21.00,TRUE,FALSE,FALSE,TRUE,2.1875000,3.000000,5.8333333
